In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large, MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mobilenet_v3_preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

2025-05-24 21:30:12.761380: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-24 21:30:12.805239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748097012.834729   15309 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748097012.847068   15309 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748097012.880702   15309 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# --- Konstanta dan Path Dataset ---
BASE_DIR = 'dataset/' # Sesuaikan jika path Anda berbeda
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TEST_DIR = os.path.join(BASE_DIR, 'test')

# --- Path Penyimpanan Hasil ---
RESULTS_BASE_DIR = 'results'
GRAPH_BASELINE_DIR = os.path.join(RESULTS_BASE_DIR, 'graph/baseline')
MODEL_BASELINE_DIR = os.path.join(RESULTS_BASE_DIR, 'model/baseline')

# Buat direktori penyimpanan jika belum ada
os.makedirs(GRAPH_BASELINE_DIR, exist_ok=True)
os.makedirs(MODEL_BASELINE_DIR, exist_ok=True)
print(f"Direktori grafik akan disimpan di: {GRAPH_BASELINE_DIR}")
print(f"Direktori model akan disimpan di: {MODEL_BASELINE_DIR}")

# --- Kelas dari dataset Anda ---
try:
    CLASSES = sorted(os.listdir(TRAIN_DIR))
    NUM_CLASSES = len(CLASSES)
    if NUM_CLASSES == 0:
        raise ValueError(f"Tidak ada direktori kelas yang ditemukan di {TRAIN_DIR}.")
    print(f"Kelas yang ditemukan: {CLASSES}")
    print(f"Jumlah kelas: {NUM_CLASSES}")
except FileNotFoundError:
    print(f"Error: Direktori '{TRAIN_DIR}' tidak ditemukan. Harap periksa path dataset Anda.")
    raise
except ValueError as ve:
    print(f"Error: {ve}")
    raise

# --- Parameter Model dan Pelatihan ---
IMG_WIDTH, IMG_HEIGHT = 224, 224
IMAGE_SIZE = (IMG_WIDTH, IMG_HEIGHT)
BATCH_SIZE = 32
EPOCHS = 50
ADAM_LR = 2e-5 # Learning rate untuk optimizer Adam

Direktori grafik akan disimpan di: results/graph/baseline
Direktori model akan disimpan di: results/model/baseline
Kelas yang ditemukan: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
Jumlah kelas: 6


In [3]:
# --- Data Generators ---
train_datagen = ImageDataGenerator(
    preprocessing_function=mobilenet_v3_preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1
)

test_datagen = ImageDataGenerator(
    preprocessing_function=mobilenet_v3_preprocess_input
)

if 'NUM_CLASSES' not in globals() or NUM_CLASSES == 0:
    raise ValueError("NUM_CLASSES tidak terdefinisi atau nol. Pastikan sel pertama berhasil dijalankan.")

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"Jumlah batch di train_generator: {len(train_generator)}")
print(f"Jumlah batch di validation_generator: {len(validation_generator)}")
print(f"Jumlah batch di test_generator: {len(test_generator)}")
print(f"Class indices train_generator: {train_generator.class_indices}")

Found 12632 images belonging to 6 classes.
Found 1402 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Jumlah batch di train_generator: 395
Jumlah batch di validation_generator: 44
Jumlah batch di test_generator: 94
Class indices train_generator: {'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}


In [4]:
# --- Fungsi untuk Plot History Pelatihan (Menyimpan ke File) ---
def plot_and_save_training_history(history, model_name_str, save_dir):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'Accuracy - {model_name_str}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title(f'Loss - {model_name_str}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.suptitle(f'Training History: {model_name_str}', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    # Simpan grafik
    file_name = f"{model_name_str.replace(' ', '_').lower()}_accuracy_loss.png"
    save_path = os.path.join(save_dir, file_name)
    plt.savefig(save_path)
    print(f"Grafik histori pelatihan disimpan ke: {save_path}")
    plt.show()

# --- Fungsi Evaluasi Umum (Menyimpan Confusion Matrix ke File) ---
def evaluate_model_and_save_cm(model, model_name_str, test_gen, class_labels, save_dir_graphs):
    print(f"\n--- Mengevaluasi {model_name_str} pada Test Set ---")

    loss, accuracy = model.evaluate(test_gen, verbose=1)
    print(f"Test Loss ({model_name_str}): {loss:.4f}")
    print(f"Test Accuracy ({model_name_str}): {accuracy:.4f}")

    test_gen.reset() # Penting untuk prediksi yang konsisten
    y_pred_proba = model.predict(test_gen, verbose=1)
    y_pred = np.argmax(y_pred_proba, axis=1)
    y_true = test_gen.classes

    print(f"\nLaporan Klasifikasi untuk {model_name_str}:")
    try:
        report = classification_report(y_true, y_pred, target_names=class_labels, zero_division=0)
        print(report)
    except ValueError as e:
        print(f"Tidak dapat menghasilkan laporan klasifikasi: {e}")

    print(f"\nConfusion Matrix untuk {model_name_str}:")
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.title(f'Confusion Matrix - {model_name_str}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    
    # Simpan confusion matrix
    file_name_cm = f"{model_name_str.replace(' ', '_').lower()}_confusion_matrix.png"
    save_path_cm = os.path.join(save_dir_graphs, file_name_cm)
    plt.savefig(save_path_cm)
    print(f"Grafik confusion matrix disimpan ke: {save_path_cm}")
    plt.show()

In [ ]:
# --- Definisi Model Variants ---
MODEL_VARIANTS = [
    ("MobileNetV3Small", MobileNetV3Small),
    ("MobileNetV3Large", MobileNetV3Large)
]

# Dapatkan label kelas sekali saja
class_labels_for_eval = list(train_generator.class_indices.keys())


for model_name_prefix, model_function in MODEL_VARIANTS:
    current_model_name = f"{model_name_prefix}_Baseline_FeatureExtract"
    print(f"\n\n{'='*30} Memproses Model: {current_model_name} {'='*30}")

    # --- 1. Load Base Model ---
    base_model = model_function(
        weights='imagenet',
        include_top=False, # Tidak menyertakan classifier head dari ImageNet
        input_shape=(IMG_WIDTH, IMG_HEIGHT, 3),
        include_preprocessing=False # Pra-pemrosesan sudah di ImageDataGenerator
    )

    # --- 2. Freeze Base Model (Tidak Ada Fine-Tuning untuk Baseline Ini) ---
    # Ini adalah strategi "feature extraction"
    base_model.trainable = False
    print(f"Semua layer di base model '{model_name_prefix}' telah dibekukan (trainable=False).")

    # --- 3. Tambahkan Layer Kustom (Classifier Head) ---
    x = base_model.output
    x = GlobalAveragePooling2D(name=f"{model_name_prefix}_gap")(x)
    x = Dense(256, activation='relu', name=f"{model_name_prefix}_dense_hidden")(x) # Opsional
    x = Dropout(0.3, name=f"{model_name_prefix}_dropout")(x) # Dropout rate bisa disesuaikan
    predictions = Dense(NUM_CLASSES, activation='softmax', name=f"{model_name_prefix}_predictions")(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # --- 4. Kompilasi Model ---
    model.compile(optimizer=Adam(learning_rate=ADAM_LR),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    print(f"\nArsitektur Model untuk {current_model_name}:")
    model.summary()

    # --- 5. Callbacks ---
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5, # Kurangi LR jika val_loss tidak membaik selama 5 epoch
        verbose=1,
        min_lr=1e-7
    )

    # --- 6. Latih Model ---
    print(f"\nMemulai pelatihan untuk {current_model_name}...")
    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=validation_generator,
        callbacks=[reduce_lr] # Tidak ada EarlyStopping
    )
    print(f"Pelatihan {current_model_name} selesai.")

    # --- 7. Visualisasi dan Simpan Hasil Pelatihan ---
    plot_and_save_training_history(history, current_model_name, GRAPH_BASELINE_DIR)

    # --- 8. Evaluasi Model pada Data Uji dan Simpan Grafik ---
    test_generator.reset() # Pastikan generator direset sebelum evaluasi
    evaluate_model_and_save_cm(model, current_model_name, test_generator, class_labels_for_eval, GRAPH_BASELINE_DIR)

    # --- 9. Simpan Model dalam Format .keras ---
    model_save_filename = f"{current_model_name.replace(' ', '_').lower()}.keras"
    model_save_path = os.path.join(MODEL_BASELINE_DIR, model_save_filename)
    model.save(model_save_path)
    print(f"Model {current_model_name} telah disimpan ke: {model_save_path}")

    print(f"\n{'='*30} Selesai memproses: {current_model_name} {'='*30}")

print("\n\nSemua model baseline telah diproses.")